<a href="https://colab.research.google.com/github/UmbertoFasci/SARIMA_SalesForecasting/blob/main/SARIMA_Retail_Sales_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forecasting monthly sales using SARIMA model

## SARIMA Models Explained

SARIMA (Seasonal Autoregressive Integrated Moving Average) is built upon the ARIMA model architecture with the addition of a seasonal component. In order to dive into what SARIMA is and what it can do, let's first explore the base ARIMA model.

ARIMA (Autoregressive Integrated Moving Average), is a model which applies (1) autoregressive methodologies to (2) orders of differencing, while (3) maintaining a rolling average. It becomes clear when separated into three components.

### 1 Autoregressive Models

Autoregressive models are a class of machine learning models that automatically predict the next value in a sequence by looking back at measurements from previous inputs in the sequence. In a time-series context, Autoregression assumes that the current value of a time series is a function of its past values.

Other applications include the enablement of generative AI, where autoregressive modeling acts as an important component of large language models, image synthesis, data augmentation, and time-series prediction.

Autoregressive models are linear and the sum of squared errors (OLS) may be utilized as a loss function not unlike linear regression and gradient descent can be additionally leveraged to determine model parameters that minimize OLS.

\begin{equation}
X_t = c + φ_{t-1} X_{t-1} + φ_{t-2} X_{t-2} + \varepsilon_t
\end{equation}

### 2 Integrated Differencing

The second component of ARIMA models is made up of the orders of differencing which calculates the diference between the adjacent target values. One order of differencing is calculted by obtaining the difference between the direct adjacent target values, while two orders of differencing is calculated using the difference between the one order of differencing values.

The purpose for the integrated differencing component of ARIMA models is to increase this magnitude of order until stationarity is achieved. Stationarity is an assumption made of time-series when implementing a moving average.  

Refer to this [ARIMA Differencing Documentation](https://people.duke.edu/~rnau/411arim2.htm) for helpful instructions on identifying the appropriate order of differencing in an ARIMA model.

### 3 Moving-Average

The assumption behind moving-average models is that time-series fluctuate randomly around a mean, and the MA model attempts to capture the stochastic nature of the time series by modeling the target as a linear combination of the mean, the current error term, and previous error terms.By using lagged error terms, the predictions incorporate the information from past errors.

\begin{equation}
X_t = \mu + \varepsilon_t + θ_{t-1 \space \epsilon t-1} + \theta_{t-2 \space  ϵ t-2}
\end{equation}

The equation above is an example of a moving average model with two lagged error terms.


Wrapping up ARIMA modeling, the model assumes that a time-series can be modeled using (1) lagged target observations and (3) lagged error terms. As mentioned before, because ARIMA models assume a stationary time series, a certain number of orders of differencing must be used until stationarity is achieved (𝑑). In this way ARIMA models, and the three components can be summarized: (𝑝,𝑑,𝑞) where 𝑝 is the number of AR terms, 𝑑 is the orders of differencing, and 𝑞 is the number of lagged error terms.


---

## The Seasonal ARIMA (SARIMA)

In order to capture the seasonality component of a time-series, we utilize a SARIMA model which has seasonal ARIMA components (𝑃,𝐷,𝑄,𝑚) in addition to the normal ARIMA components (𝑝,𝑑,𝑞). 𝑚 represents the number of time periods in a season. If forecasting **monthly** sales then 𝑚 = 12, and 𝑃, 𝐷, 𝑄 are the same as 𝑝, 𝑑, 𝑞 except the former terms are using seasonal lags based on what 𝑚 is.

In this case, if 𝑃 = 2, then there are 2 seasonal autoregressive terms where the first term refers to the target observation 12 months ago and the second term refers to the target observation 24 months ago.

Overall, the components the make up a SARIMA model can be summarized: (𝑝,𝑑,𝑞)(𝑃,𝐷,𝑄,𝑚).

## Data Exploratory Analysis

The data for this project was gathered from the UCI Machine Learning Repository - Online Retail I data source.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
sales_df = pd.read_csv('/content/drive/MyDrive/SARIMA_Forecasting/data/OnlineRetail.csv', parse_dates=['InvoiceDate'], infer_datetime_format=True)

/tmp/ipython-input-3804618995.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  sales_df = pd.read_csv('/content/drive/MyDrive/SARIMA_Forecasting/data/OnlineRetail.csv', parse_dates=['InvoiceDate'], infer_datetime_format=True)


In [4]:
sales_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [6]:
sales_df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [7]:
sales_df.describe(include='object')

,InvoiceNo,StockCode,Description,Country
count,541909,541909,540455,541909
unique,25900,4070,4223,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
freq,1114,2313,2369,495478


### Missing Value Handling

Having an initial look at the data, the most concerning missing value distribution is concentrated with the `CustomerID` feature which I expect to limit the analysis to the extent of its missingness.

Let's have a closer look at what the data looks like at these points of missingness.

In [9]:
sales_df[sales_df['CustomerID'].isna()].head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom
1447,536544,21790,VINTAGE SNAP CARDS,9,2010-12-01 14:32:00,1.66,NaN,United Kingdom
1448,536544,21791,VINTAGE HEADS AND TAILS CARD GAME,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1449,536544,21801,CHRISTMAS TREE DECORATION WITH BELL,10,2010-12-01 14:32:00,0.43,NaN,United Kingdom
1450,536544,21802,CHRISTMAS TREE HEART DECORATION,9,2010-12-01 14:32:00,0.43,NaN,United Kingdom
1451,536544,21803,CHRISTMAS TREE STAR DECORATION,11,2010-12-01 14:32:00,0.43,NaN,United Kingdom


Since there is no reasonable way to assume the identity of these customers, we can simply drop the missing `CustomerID` values. The missing `Description` values are not too important for the purposes of this procedure, but let's still take a look at how the data looks at those points as well.

In [10]:
sales_df[sales_df['Description'].isna()].head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
1988,536550,85044,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
2024,536552,20950,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
2025,536553,37461,NaN,3,2010-12-01 14:35:00,0.0,NaN,United Kingdom
2026,536554,84670,NaN,23,2010-12-01 14:35:00,0.0,NaN,United Kingdom
2406,536589,21777,NaN,-10,2010-12-01 16:50:00,0.0,NaN,United Kingdom


Do all null `Descriptions` also have null `CustomerID`?

In [13]:
# True: `CustomerID` values exist on entries with missing `Description`
# False: All entries with null `Description` also have null `CustomerID`
description_null_customer_id = sales_df[sales_df['Description'].isna()]['CustomerID'].notna().any()

description_null_customer_id

np.False_

From that we can see that by handling the `CustomerID` column by dropping all missing values, the missing values in the `Description` feature will also be handled.

In the real-world, it is recommended to provide a missing value analysis - Asking why these values are missing and what that means for the original source data.

For example, for entry **622** where quantity = 56. This might be a bulk custom order for a specific distributor or partner store and the product is not publically available online, thus no real product was created for it. From my experience these usually represent ad-hoc product creations that are not completed through the product creation system, but through the product sales system instead.

In [14]:
sales_df.dropna(subset=['CustomerID'], inplace=True)

### Negative Quantity Values

Earlier in the analysis, negative instances of `Quantity` can be noted. What that means will be explored in this section.

In [16]:
sales_df[sales_df['Quantity'] < 0].head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
239,C536391,21484,CHICK GREY HOT WATER BOTTLE,-12,2010-12-01 10:24:00,3.45,17548.0,United Kingdom
240,C536391,22557,PLASTERS IN TIN VINTAGE PAISLEY,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
241,C536391,22553,PLASTERS IN TIN SKULLS,-24,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
939,C536506,22960,JAM MAKING SET WITH JARS,-6,2010-12-01 12:38:00,4.25,17897.0,United Kingdom


With the given data description which can be found [here](https://archive.ics.uci.edu/dataset/352/online+retail), a 'C' at the front of the of the invoice number represents orders which have been cancelled, and much of these negative quantities are cancelled orders. For the purposes of this procedure, negative quantity values representing canceled orders will be dropped from the dataset.

In [17]:
sales_df = sales_df[sales_df['Quantity'] > 0]

On this note of Invoice Number codes, let's see if there are any additional ones besides 'C'. With a bit of regex magic we can remove all the numbers from the Invoice values and return only the letters. Since we dropped all negative quantities, we should expect for 'C' to be absent from this list.

In [23]:
sales_df['InvoiceNo'].str.replace("[0-9]", "", regex=True).unique()

array([''], dtype=object)

'C' is absent, and there are no additional invoice descriptors besides those that described the already removed cancelled orders.